In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os



In [2]:
data_path = os.path.join("..", "data")

In [3]:
X = np.load(data_path + '/initial_data/function_5/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_5/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.19144708 0.03819337 0.60741781 0.41458414]
 [0.75865295 0.53651774 0.65600038 0.36034155]
 [0.43834987 0.8043397  0.21024527 0.15129482]
 [0.70605083 0.53419196 0.26424335 0.48208755]
 [0.83647799 0.19360965 0.6638927  0.78564888]
 [0.68343225 0.11866264 0.82904591 0.56757661]
 [0.55362148 0.66734998 0.32380582 0.81486975]
 [0.35235627 0.32224153 0.11697937 0.47311252]
 [0.15378571 0.72938169 0.42259844 0.44307417]
 [0.46344227 0.63002451 0.10790646 0.9576439 ]
 [0.67749115 0.35850951 0.47959222 0.07288048]
 [0.58397341 0.14724265 0.34809746 0.42861465]
 [0.30688872 0.31687813 0.62263448 0.09539906]
 [0.51114177 0.817957   0.72871042 0.11235362]
 [0.43893338 0.77409176 0.37816709 0.93369621]
 [0.22418902 0.84648049 0.87948418 0.87851568]
 [0.72526172 0.47987049 0.08894684 0.75976022]
 [0.35548161 0.63961937 0.41761768 0.12260384]
 [0.11987923 0.86254031 0.64333133 0.84980383]
 [0.12688467 0.15342962 0.77016219 0.19051811]]
Y is :[6.44434399e+01 1.83013796e+01 1.12939795e-01 4.

In [4]:
# add the week 1 input and  result to the array
X = np.append(X, [[0.224489, 0.836734, 0.877551, 0.877551]], axis=0)
y = np.append(y, [1035.7833689308636], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.204082, 0.877551, 0.857143, 0.897959]], axis=0)
y = np.append(y, [1221.5705853873078], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.220562, 0.993718, 0.923250, 0.932260]], axis=0)
y = np.append(y, [2763.136135055027], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.210582, 0.914111, 0.854212, 0.943472]], axis=0)
y = np.append(y, [1699.4414716139875], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.223249, 0.991019, 0.987207, 0.989510]], axis=0)
y = np.append(y, [4021.5011695428084], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.210130, 0.932370, 0.868367, 0.947575]], axis=0)
y = np.append(y, [1938.9432672210037], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.256514, 0.997140, 0.999654, 0.992373]], axis=0)
y = np.append(y, [4310.980559441547], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

X is:[[0.19144708 0.03819337 0.60741781 0.41458414]
 [0.75865295 0.53651774 0.65600038 0.36034155]
 [0.43834987 0.8043397  0.21024527 0.15129482]
 [0.70605083 0.53419196 0.26424335 0.48208755]
 [0.83647799 0.19360965 0.6638927  0.78564888]
 [0.68343225 0.11866264 0.82904591 0.56757661]
 [0.55362148 0.66734998 0.32380582 0.81486975]
 [0.35235627 0.32224153 0.11697937 0.47311252]
 [0.15378571 0.72938169 0.42259844 0.44307417]
 [0.46344227 0.63002451 0.10790646 0.9576439 ]
 [0.67749115 0.35850951 0.47959222 0.07288048]
 [0.58397341 0.14724265 0.34809746 0.42861465]
 [0.30688872 0.31687813 0.62263448 0.09539906]
 [0.51114177 0.817957   0.72871042 0.11235362]
 [0.43893338 0.77409176 0.37816709 0.93369621]
 [0.22418902 0.84648049 0.87948418 0.87851568]
 [0.72526172 0.47987049 0.08894684 0.75976022]
 [0.35548161 0.63961937 0.41761768 0.12260384]
 [0.11987923 0.86254031 0.64333133 0.84980383]
 [0.12688467 0.15342962 0.77016219 0.19051811]
 [0.224489   0.836734   0.877551   0.877551  ]
 [0.2040

In [5]:
print(f"max y is :{np.max(y)}")

max y is :4310.980559441547


In [6]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

GaussianProcessRegressor(kernel=Matern(length_scale=1, nu=2.5),
                         normalize_y=True)

In [7]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 7 changed kappa to 1.0 from 4.0 for exploitation
# week 8 changed kappa to 0.5 for more exploitation
def ucb(x, gp, kappa=0.5):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
"""grid_size = 50
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
x3 = np.linspace(0, 1, grid_size)
x4 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b, c, d] for a in x1 for b in x2 for c in x3 for d in x4])"""

In [8]:
n_samples = 3000000 # You can adjust this based on your compute budget

# Generate random 6D points in [0, 1]^6
X_random = np.random.rand(n_samples, 4)

In [9]:
ucb_values = np.array([ucb(x, gp) for x in X_random])


In [10]:
best_idx = np.argmax(ucb_values)
next_best_point = X_random[best_idx]
print("Next best point to evaluate (UCB) rounded:", np.round(next_best_point,6))
print("Next best point to evaluate (UCB):", next_best_point)
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])

Next best point to evaluate (UCB) rounded: [0.331337 0.998687 0.980926 0.995698]
Next best point to evaluate (UCB): [0.33133734 0.99868701 0.98092595 0.99569794]
Predicted maximum value (mean): 4196.196036389948


In [15]:
# SVM model as surrogate
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=5000.0, epsilon=2.0)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 4)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]

print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)

Best candidate input: [0.26797866 0.99512174 0.99965098 0.99123052]
Best candidate input rounded: [0.267979 0.995122 0.999651 0.991231]
Predicted maximum value: 3327.9711854080533
